Desktop
    datafile
        NFT_All.csv
        NFT_emotion.csv
        positive_or_negative.csv

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import opinion_lexicon
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.corpus import stopwords
import string
import seaborn as sns
from nltk import FreqDist

In [2]:
# Import the file to be read
df = pd.read_spss(r"/Users/Desktop/datafile/NFT_All.sav")

# 1. The frequency of each person's emotional choices

In [61]:
df1 = pd.DataFrame(df.iloc[:,15:733:3].apply(lambda x:x.value_counts().to_dict(),axis=1))

In [62]:
def select(cell,emo):
    if emo in cell.keys():
        return cell[emo]
    else:
        return 

In [63]:
df["if_disgust"] = df1.applymap(lambda x:select(x,emo="Disgust"))

In [64]:
df["if_Amusement"] = df1.applymap(lambda x:select(x,emo="Amusement"))
df["if_Awe"] = df1.applymap(lambda x:select(x,emo="Awe"))
df["if_Contentment"] = df1.applymap(lambda x:select(x,emo="Contentment"))
df["if_Excitement"] = df1.applymap(lambda x:select(x,emo="Excitement"))
df["if_Anger"] = df1.applymap(lambda x:select(x,emo="Anger"))
df["if_Fear"] = df1.applymap(lambda x:select(x,emo="Fear"))
df["if_Sandness"] = df1.applymap(lambda x:select(x,emo="Sadness"))
df["other"] =df1.applymap(lambda x:select(x,emo="Something else（Please specify which emotion it is below）"))

In [65]:
# export data
df.to_csv(r"/Users/Desktop/datafile/1_expert.csv",encoding="gbk")

# 2. Count the number of choices for each emotion by item

In [106]:
temp = pd.read_spss(r"/Users/Desktop/datafile/NFT_emotion.sav")

In [107]:
temp =temp.iloc[:,777:885]

In [108]:
result = []
columns_name_list = [["amusement","awe","contenment","excitement","anger","disgust","fear","sandnedd","other"]]
for i in range(0,len(temp.columns),9):
    df_temp = temp[ 
        temp.iloc[:,i:i+9].notna().any(axis=1)
    ].iloc[:,i:i+9]
    result.append(df_temp.sum().to_list())

result_df  = pd.DataFrame(result)
result_df.columns=[["amusement","awe","contenment","excitement","anger","disgust","fear","sandnedd","other"]]
result_df.index=[i for i in range(1,13)]

In [109]:
result_df.to_csv(r"/Users/Desktop/datafile/2_expert.csv")

# 3. The text of emotion interpretation is divided into words and word frequency is extracted

In [13]:
df_cipin = df.iloc[:,17:737:3]

In [14]:
sentence_list = list(df_cipin.to_numpy().flatten()) 
sentence_list = [x for x in sentence_list if len(x)!=0] 
sentence_str = ".".join(sentence_list) 
pd.DataFrame(sentence_list).to_csv(r"/Users/Desktop/datafile/3_expert.csv",encoding="GBK")

In [15]:
sentence_str = " ".join(sentence_list)

In [16]:
def filter_punctuation(text):
    punctuations = string.punctuation
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in punctuations]
    return filtered_tokens



token = filter_punctuation(sentence_str)

In [17]:
token = [word for word  in token if word.lower() not in stopwords.words("english")]

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# 1. Acquired part of speech
def get_wordnet_pos(tag): 
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# 2. Part of speech merging
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemmatized_token = lemmatizer.lemmatize(token, pos=pos)
        lemmatized_tokens.append(lemmatized_token)
    return lemmatized_tokens



lem_result = lemmatize_tokens(token)


In [1]:
t = Text(lem_result)
t.plot(30)

NameError: name 'Text' is not defined

## 4. Output word frequency and part of speech

In [118]:
freq = FreqDist(lem_result)
pd.DataFrame(
   {"word":freq.keys(),
    "freq":freq.values(),
    "type":[i[1] for i in  pos_tag(freq.keys())]
    }
).to_csv(r"/Users/Desktop/datafile/4_expert.csv")

# 5. Eight key words affective tendency

In [276]:

df_image  = pd.read_spss(r"/Users/Desktop/datafile/NFT_All.sav").iloc[:,15:735]


In [277]:

for i in df_image.columns:
    if "Other" in i:
        df_image.drop(labels=i,axis=1,inplace=True)

In [278]:
def func(x):
    x = x.replace("color","colour").replace("Colours","colour").replace("colourful","colour").replace("colouring","colour").replace("multi-coloured","colour").replace("Color","colour").replace("colorful","colour").replace("EXPRESSION","expression").replace("eyes","eye").replace("EYES","eye").replace("Color","colour").replace("colorful","colour") .replace("EXPRESSION","expression").replace("eyes","eye").replace("skin","reskin").replace("hats","hat").replace("expressioj","expression").replace("hairs","hair").replace("mouth/nose","other").replace("mouthpiece","other")


    return x

In [279]:
df_image.iloc[:,1::2] = df_image.iloc[:,1::2].applymap(lambda x:func(x))

In [280]:

image_list = []
i=0
while i<len(df_image.columns) :
    if i%2==0:
        temp = df_image.iloc[:,i:i+2]
        image_list.append(temp)
    i += 1

In [281]:
keywords = ["colour","expression","eye","mouth","hair","hat","skin","background"]

In [282]:


def keyword_func(cell,keyword):
    return keyword in cell
    

def calculate(x):
    temp_list = []
    for df  in x:
        res = df[df.iloc[:,1] == True].iloc[:,0].value_counts()
        temp_list.append(res)
    return temp_list

In [283]:
# Exporting different keyword sentiment data needs to be adjusted in this line of code
df_image.iloc[:,1::2] = df_image.iloc[:,1::2].applymap(lambda x: keyword_func(x,"mouth"))

/var/folders/y8/mzg2yr0d4kg8xhb8s78tgd8w0000gn/T/ipykernel_10190/4238437664.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_image.iloc[:,1::2] = df_image.iloc[:,1::2].applymap(lambda x: keyword_func(x,"mouth"))


In [284]:

i = 0
data_list  = []
while i< len(df_image.columns):
    if i%2==0:
       data_list.append(df_image.iloc[:,i:i+2])
    i+=1

In [285]:

pd.DataFrame(calculate(data_list)).fillna(value=0).to_csv(r"/Users/Desktop/datafile/mouth.csv",encoding="GBK")

# 6. Judgment of positive/negative emotions

In [2]:
df = pd.read_spss(r"/Users/Desktop/datafile/NFT_All.sav")

In [3]:
df1 = df.iloc[:,15:734:3]

In [4]:
df1 = df1.replace({
    "Amusement":1,
    "Awe":1,
    "Contentment":1,
    "Excitement":1,
    "Anger":-1,
    "Disgust":-1,
    "Fear":-1,
    "Sadness":-1,
    "Something else（Please specify which emotion it is below）":0,
    "nan":0


})

In [5]:
df1

,Image_1,Image_2,Image_3,Image_4,Image_5,Image_6,Image_7,Image_8,Image_9,Image_10,...,Image_231,Image_232,Image_233,Image_234,Image_235,Image_236,Image_237,Image_238,Image_239,Image_240
0,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df1.to_csv(r"/Users/Desktop/datafile/6_expert.csv",encoding="gbk")